In [1]:
import kfp, os
import kfp.dsl as dsl 
import kfp.compiler as compiler
import kubernetes.client.models as k8s
import namesgenerator

In [2]:
NAMESPACE = os.environ.get("NAMESPACE") 

In [3]:
kubeflow_address = f"http://{NAMESPACE}.kubeflow.odsc.k8s.hydrosphere.io"
hydrosphere_address = f"http://{NAMESPACE}.serving.odsc.k8s.hydrosphere.io"

In [4]:
!docker login

Authenticating with existing credentials...
Login Succeeded


# Download

At this stage we will obtain all training data for our pipeline.

Build & publish an image

In [5]:
%%bash 
docker build -t tidylobster/mnist-pipeline-download:latest --no-cache 01_download
docker push tidylobster/mnist-pipeline-download:latest


Step 1/4 : FROM tidylobster/odsc-base:1.0
1.0: Pulling from tidylobster/odsc-base
7b722c1070cd: Already exists
5fbf74db61f1: Already exists
ed41cb72e5c9: Already exists
7ea47a67709e: Already exists
b961d2edf417: Already exists
ebdeec4d2f68: Already exists
575aab5f7ff1: Already exists
c5d5adaa6198: Already exists
707316736cfa: Already exists
3fce84e77cad: Already exists
c40ccc181cbc: Pulling fs layer
3d825a728840: Pulling fs layer
c40ccc181cbc: Download complete
c40ccc181cbc: Pull complete
3d825a728840: Download complete
3d825a728840: Pull complete
Digest: sha256:8f50b0a7ac201e9ecb2e285aa3bf94af9144bc63d26700fc9f4329a6d405b98f
Status: Downloaded newer image for tidylobster/odsc-base:1.0
 ---> a44d37e5b862
Step 2/4 : ADD ./download.py /src/
 ---> 642b1cf36351
Step 3/4 : WORKDIR /src/
 ---> Running in 72bf14e70a4d
Removing intermediate container 72bf14e70a4d
 ---> 410aecc0943c
Step 4/4 : ENTRYPOINT [ "python", "download.py" ]
 ---> Running in 32e6819380ce
Removing intermediate container 

### Pipeline

Create Kubernetes PVC resource

In [5]:
storage_pvc = k8s.V1PersistentVolumeClaimVolumeSource(claim_name="storage")
storage_volume = k8s.V1Volume(name="storage", persistent_volume_claim=storage_pvc)
storage_volume_mount = k8s.V1VolumeMount(mount_path="{{workflow.parameters.mount-path}}", name="storage")

Define container operation

In [6]:
username = "tidylobster"

In [7]:
def download_op(**kwargs):
    download = dsl.ContainerOp(
        name="download",
        image=f"{username}/mnist-pipeline-download:latest",  # <-- Replace with built docker image
        file_outputs={"data_path": "/data_path.txt"},
        arguments=["--mount-path", kwargs["mount_path"]])
    
    download.add_volume(storage_volume)
    download.add_volume_mount(storage_volume_mount)
    return download

In [8]:
@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(
    mount_path="/storage",
):
    download = download_op(
        mount_path=mount_path)

In [9]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

In [10]:
%%bash 

tar -xvf pipeline.tar.gz
sed -i '' s/minio-service.kubeflow/minio-service.${NAMESPACE}/g pipeline.yaml
sed -i '' s/pipeline-runner/${NAMESPACE}-pipeline-runner/g pipeline.yaml

x pipeline.yaml


### Test

In [11]:
# Create Pipelines client
client = kfp.Client(kubeflow_address)

In [12]:
# Define an experiment name
experiment_name='MNIST Showreal'

In [13]:
# get or create an experiment_id
try:
    experiment_id = client.get_experiment(experiment_name=experiment_name).id
except:
    experiment_id = client.create_experiment(experiment_name).id

In [14]:
# start a run
run_name = namesgenerator.get_random_name()
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(experiment_id, run_name, "pipeline.yaml")

Starting a new run with the name nervous_hamilton


ApiException: (500)
Reason: Internal Server Error
HTTP response headers: HTTPHeaderDict({'Server': 'nginx/1.15.3', 'Date': 'Fri, 26 Apr 2019 12:13:27 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'X-Powered-By': 'Express', 'trailer': 'Grpc-Trailer-Content-Type'})
HTTP response body: {"error":"Failed to create a new run.: InternalServerError: Failed to create a workflow for (): Unauthorized","message":"Failed to create a new run.: InternalServerError: Failed to create a workflow for (): Unauthorized","code":13,"details":[{"@type":"type.googleapis.com/api.Error","error_message":"Internal Server Error","error_details":"Failed to create a new run.: InternalServerError: Failed to create a workflow for (): Unauthorized"}]}


# Train

At this stage we will create a model & train it on the downloaded data.

Build and publish image

In [15]:
%%bash 
docker build -t tidylobster/mnist-pipeline-train:latest --no-cache 02_train/ 
docker push tidylobster/mnist-pipeline-train:latest


Step 1/4 : FROM tidylobster/odsc-base:1.0
 ---> a44d37e5b862
Step 2/4 : ADD ./train.py /src/
 ---> 6d6d3ee05d63
Step 3/4 : WORKDIR /src/
 ---> Running in 04dc3e9b4a8d
Removing intermediate container 04dc3e9b4a8d
 ---> 3f1cf427c4d5
Step 4/4 : ENTRYPOINT [ "python", "train.py" ]
 ---> Running in b6887fdfdfce
Removing intermediate container b6887fdfdfce
 ---> a5a20bfbdf98
Successfully built a5a20bfbdf98
Successfully tagged tidylobster/mnist-pipeline-train:latest
The push refers to repository [docker.io/tidylobster/mnist-pipeline-train]
2f081c0fe8f5: Preparing
66a75017de07: Preparing
83c720aa6f39: Preparing
56995c671038: Preparing
eee35c27cf87: Preparing
93ed1238773c: Preparing
eeb5ce6b3db4: Preparing
886601877ba4: Preparing
0fc100fdc7f9: Preparing
68dda0c9a8cd: Preparing
f67191ae09b8: Preparing
b2fd8b4c3da7: Preparing
0de2edf7bff4: Preparing
93ed1238773c: Waiting
eeb5ce6b3db4: Waiting
886601877ba4: Waiting
0fc100fdc7f9: Waiting
68dda0c9a8cd: Waiting
f67191ae09b8: Waiting
b2fd8b4c3da7: Wa

### Pipeline

Define container operation

In [14]:
def train_op(download, **kwargs):
    train = dsl.ContainerOp(
        name="train",
        image=f"{username}/mnist-pipeline-train:latest",        # <-- Replace with correct docker image
        file_outputs={"accuracy": "/accuracy.txt"},
        arguments=[
            "--data-path", download.outputs["data_path"], 
            "--mount-path", kwargs["mount_path"],
            "--learning-rate", kwargs["learning_rate"],
            "--epochs", kwargs["epochs"],
            "--batch-size", kwargs["batch_size"]
        ])

    train.add_volume(storage_volume)
    train.add_volume_mount(storage_volume_mount)
    return train

In [15]:
@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(
    mount_path="/storage",
    learning_rate="0.01",
    epochs="10",
    batch_size="256",
):
    
    download = download_op(
        mount_path=mount_path)
    
    train = train_op(
        download, 
        mount_path=mount_path, 
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=batch_size)
    train.after(download)
    train.set_memory_request('1G')
    train.set_cpu_request('1')

In [16]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

In [17]:
%%bash 

tar -xvf pipeline.tar.gz
sed -i '' s/minio-service.kubeflow/minio-service.${NAMESPACE}/g pipeline.yaml
sed -i '' s/pipeline-runner/${NAMESPACE}-pipeline-runner/g pipeline.yaml

x pipeline.yaml


### Test

In [18]:
# start a run
run_name = namesgenerator.get_random_name()
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(
    experiment_id, run_name, "pipeline.yaml",
    {
        "learning-rate": "0.01",
        "batch-size": "256",
        "epochs": "1"
    }
)

Starting a new run with the name sad_dijkstra


ApiException: (500)
Reason: Internal Server Error
HTTP response headers: HTTPHeaderDict({'Server': 'nginx/1.15.3', 'Date': 'Fri, 26 Apr 2019 12:10:42 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'X-Powered-By': 'Express', 'trailer': 'Grpc-Trailer-Content-Type'})
HTTP response body: {"error":"Failed to create a new run.: InternalServerError: Failed to create a workflow for (): Unauthorized","message":"Failed to create a new run.: InternalServerError: Failed to create a workflow for (): Unauthorized","code":13,"details":[{"@type":"type.googleapis.com/api.Error","error_message":"Internal Server Error","error_details":"Failed to create a new run.: InternalServerError: Failed to create a workflow for (): Unauthorized"}]}


# Release

At this stage we will release the trained model to the cluster

Build and publish image

In [18]:
%%bash 
docker build -t tidylobster/mnist-pipeline-release:latest --no-cache 03_release/ 
docker push tidylobster/mnist-pipeline-release:latest


Step 1/4 : FROM tidylobster/odsc-base:1.0
 ---> a44d37e5b862
Step 2/4 : ADD ./execute.sh /src/
 ---> 6a00f6afe7d0
Step 3/4 : WORKDIR /src/
 ---> Running in 3efab383681b
Removing intermediate container 3efab383681b
 ---> 5f33f8fa38cd
Step 4/4 : ENTRYPOINT [ "bash", "execute.sh" ]
 ---> Running in dc8fcaad7f8d
Removing intermediate container dc8fcaad7f8d
 ---> e40a5e8e1abb
Successfully built e40a5e8e1abb
Successfully tagged tidylobster/mnist-pipeline-release:latest
The push refers to repository [docker.io/tidylobster/mnist-pipeline-release]
2e85e0f8fb5c: Preparing
66a75017de07: Preparing
83c720aa6f39: Preparing
56995c671038: Preparing
eee35c27cf87: Preparing
93ed1238773c: Preparing
eeb5ce6b3db4: Preparing
886601877ba4: Preparing
0fc100fdc7f9: Preparing
68dda0c9a8cd: Preparing
f67191ae09b8: Preparing
b2fd8b4c3da7: Preparing
0de2edf7bff4: Preparing
eeb5ce6b3db4: Waiting
0fc100fdc7f9: Waiting
68dda0c9a8cd: Waiting
f67191ae09b8: Waiting
b2fd8b4c3da7: Waiting
886601877ba4: Waiting
93ed123877

### Pipeline

Define container operation

In [13]:
def release_op(download, train, **kwargs):
    release = dsl.ContainerOp(
        name="release",
        image=f"{username}/mnist-pipeline-release:latest",        # <-- Replace with correct docker image
        file_outputs={"model-version": "/model-version.txt"},
        arguments=[
            "--data-path", download.outputs["data_path"],
            "--mount-path", kwargs["mount_path"],
            "--accuracy", train.outputs["accuracy"], 
            "--hydrosphere-address", kwargs["hydrosphere_address"],
            "--learning-rate", kwargs["learning_rate"],
            "--epochs", kwargs["epochs"],
            "--batch-size", kwargs["batch_size"],
        ])

    release.add_volume(storage_volume) 
    release.add_volume_mount(storage_volume_mount)
    return release

In [25]:
@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(
    mount_path="/storage",
    learning_rate="0.01",
    epochs="10",
    batch_size="256",
    model_name="mnist",
    hydrosphere_address=""
):
    
    download = download_op(
        mount_path=mount_path)
    
    train = train_op(
        download, 
        mount_path=mount_path, 
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=batch_size)
    train.after(download)
    train.set_memory_request('1G')
    train.set_cpu_request('1')
    
    release = release_op(
        download, 
        train
        mount_path=mount_path, 
        hydrosphere_address=hydrosphere_address,
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=batch_size)
    release.after(train)

In [26]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

In [27]:
%%bash 

tar -xvf pipeline.tar.gz
sed -i '' s/minio-service.kubeflow/minio-service.${NAMESPACE}/g pipeline.yaml
sed -i '' s/pipeline-runner/${NAMESPACE}-pipeline-runner/g pipeline.yaml

x pipeline.yaml


### Test

In [28]:
# start a run
run_name = namesgenerator.get_random_name()
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(
    experiment_id, run_name, "pipeline.yaml",
    {
        "learning-rate": "0.01",
        "batch-size": "256",
        "epochs": "10",
        "hydrosphere-address": hydrosphere_address,
    }
)

Starting a new run with the name elastic_fermi


# Deploy to Stage

At this stage we are deploying the model to the stage application to run integration tests.

Build and publish image

In [19]:
%%bash 
docker build -t tidylobster/mnist-pipeline-deploy-to-stage:latest --no-cache 04_deploy-to-stage/ 
docker push tidylobster/mnist-pipeline-deploy-to-stage:latest


Step 1/4 : FROM tidylobster/odsc-base:1.0
 ---> a44d37e5b862
Step 2/4 : ADD ./execute.sh /src/
 ---> 598ffa5d76b5
Step 3/4 : WORKDIR /src/
 ---> Running in aad2fdaf6e71
Removing intermediate container aad2fdaf6e71
 ---> 6ff09d10870d
Step 4/4 : ENTRYPOINT [ "bash", "execute.sh" ]
 ---> Running in 192af4c52c68
Removing intermediate container 192af4c52c68
 ---> a3d9daef6a25
Successfully built a3d9daef6a25
Successfully tagged tidylobster/mnist-pipeline-deploy-to-stage:latest
The push refers to repository [docker.io/tidylobster/mnist-pipeline-deploy-to-stage]
ee538f44b6d8: Preparing
66a75017de07: Preparing
83c720aa6f39: Preparing
56995c671038: Preparing
eee35c27cf87: Preparing
93ed1238773c: Preparing
eeb5ce6b3db4: Preparing
886601877ba4: Preparing
0fc100fdc7f9: Preparing
68dda0c9a8cd: Preparing
f67191ae09b8: Preparing
b2fd8b4c3da7: Preparing
0de2edf7bff4: Preparing
0fc100fdc7f9: Waiting
68dda0c9a8cd: Waiting
eeb5ce6b3db4: Waiting
f67191ae09b8: Waiting
b2fd8b4c3da7: Waiting
0de2edf7bff4: Wa

### Pipeline

Create required environmnet variables

In [14]:
application_name_env = k8s.V1EnvVar(name="APPLICATION_NAME", value="{{workflow.parameters.application-name}}")

Define container operation

In [15]:
def deploy_to_stage_op(release, **kwargs):
    deploy_to_stage = dsl.ContainerOp(
        name="deploy_to_stage",
        image=f"{username}/mnist-pipeline-deploy-to-stage:latest",        # <-- Replace with correct docker image
        file_outputs={"stage-app-name": "/stage-app-name.txt"},
        arguments=[
            "--model-version", release.outputs["model-version"],
            "--hydrosphere-address", kwargs["hydrosphere_address"],
            "--model-name", kwargs["model_name"],
        ],
    )
    
    return deploy_to_stage

In [32]:
@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(
    mount_path="/storage",
    learning_rate="0.01",
    epochs="10",
    batch_size="256",
    model_name="mnist",
    hydrosphere_address="",
):
    
    download = download_op(
        mount_path=mount_path)
    
    train = train_op(
        download, 
        mount_path=mount_path, 
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=batch_size)
    train.after(download)
    train.set_memory_request('1G')
    train.set_cpu_request('1')
    
    release = release_op(
        download, 
        train
        mount_path=mount_path, 
        hydrosphere_address=hydrosphere_address,
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=batch_size)
    release.after(train)
    
    deploy_to_stage = deploy_to_stage_op(
        release
        hydrosphere_address=hydrosphere_address,
        model_name=model_name)
    deploy_to_stage.after(release)


In [33]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

In [34]:
%%bash 

tar -xvf pipeline.tar.gz
sed -i '' s/minio-service.kubeflow/minio-service.${NAMESPACE}/g pipeline.yaml
sed -i '' s/pipeline-runner/${NAMESPACE}-pipeline-runner/g pipeline.yaml

x pipeline.yaml


### Test

In [35]:
# start a run
run_name = namesgenerator.get_random_name()
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(
    experiment_id, run_name, "pipeline.yaml",
    {
        "learning-rate": "0.01",
        "batch-size": "256",
        "epochs": "10",
        "hydrosphere-address": hydrosphere_address,
    }
)

Starting a new run with the name optimistic_leavitt


# Test

At this stage we are performing integration tests.

Build and publish image

In [36]:
%%bash 
docker build -t tidylobster/mnist-pipeline-test:latest --no-cache 05_test/ 
docker push tidylobster/mnist-pipeline-test:latest


Step 1/4 : FROM tidylobster/odsc-base:1.0
 ---> a44d37e5b862
Step 2/4 : ADD ./test.py /src/
 ---> 045e1ae7d4d4
Step 3/4 : WORKDIR /src/
 ---> Running in 1f9ca431d3d6
Removing intermediate container 1f9ca431d3d6
 ---> ac29e11563c5
Step 4/4 : ENTRYPOINT [ "python", "test.py" ]
 ---> Running in dddc68c4da11
Removing intermediate container dddc68c4da11
 ---> dda556327cc5
Successfully built dda556327cc5
Successfully tagged tidylobster/mnist-pipeline-test:latest
The push refers to repository [docker.io/tidylobster/mnist-pipeline-test]
61de64499d97: Preparing
66a75017de07: Preparing
83c720aa6f39: Preparing
56995c671038: Preparing
eee35c27cf87: Preparing
93ed1238773c: Preparing
eeb5ce6b3db4: Preparing
886601877ba4: Preparing
0fc100fdc7f9: Preparing
68dda0c9a8cd: Preparing
f67191ae09b8: Preparing
b2fd8b4c3da7: Preparing
0de2edf7bff4: Preparing
886601877ba4: Waiting
0fc100fdc7f9: Waiting
68dda0c9a8cd: Waiting
f67191ae09b8: Waiting
b2fd8b4c3da7: Waiting
0de2edf7bff4: Waiting
eeb5ce6b3db4: Waitin

### Pipeline

Create required environmnet variables

In [16]:
test_amount_env = k8s.V1EnvVar(name="TEST_AMOUNT", value="{{workflow.parameters.test-amount}}")
requests_delay_env = k8s.V1EnvVar(name="REQUESTS_DELAY", value="{{workflow.parameters.requests-delay}}")
acceptable_accuracy_env = k8s.V1EnvVar(name="ACCEPTABLE_ACCURACY", value="{{workflow.parameters.acceptable-accuracy}}")

Define container operation

In [17]:
def test_op(download, stage, **kwargs):
    test = dsl.ContainerOp(
        name="test",
        image=f"{username}/mnist-pipeline-test:latest",               # <-- Replace with correct docker image
        arguments=[
            "--stage-app-name", stage.outputs["stage-app-name"], 
            "--data-path", download.outputs["data_path"],
            "--mount-path", kwargs["mount_path"],
            "--hydrosphere-address", kwargs["hydrosphere_address"],
            "--acceptable-acuracy", kwargs["acceptable_accuracy"],
            "--model-name", kwargs["model_name"], 
        ],
    )

    test.add_volume(storage_volume) 
    test.add_volume_mount(storage_volume_mount)
    return test

In [39]:
@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(
    mount_path="/storage",
    learning_rate="0.01",
    epochs="10",
    batch_size="256",
    model_name="mnist",
    hydrosphere_address="",
    acceptable_accuracy="0.90",
):
    
    download = download_op(
        mount_path=mount_path)
    
    train = train_op(
        download, 
        mount_path=mount_path, 
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=batch_size)
    train.after(download)
    train.set_memory_request('1G')
    train.set_cpu_request('1')
    
    release = release_op(
        download, 
        train
        mount_path=mount_path, 
        hydrosphere_address=hydrosphere_address,
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=batch_size)
    release.after(train)
    
    deploy_to_stage = deploy_to_stage_op(
        release
        hydrosphere_address=hydrosphere_address,
        model_name=model_name)
    deploy_to_stage.after(release)
    
    test = test_op(
        download, 
        deploy_to_stage,
        mount_path=mount_path,
        hydrosphere_address=hydrosphere_address,
        acceptable_accuracy=acceptable_accuracy,
        model_name=model_name)
    test.set_retry(3)
    test.after(deploy_to_stage)

In [40]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

In [41]:
%%bash 

tar -xvf pipeline.tar.gz
sed -i '' s/minio-service.kubeflow/minio-service.${NAMESPACE}/g pipeline.yaml
sed -i '' s/pipeline-runner/${NAMESPACE}-pipeline-runner/g pipeline.yaml

x pipeline.yaml


### Test

In [42]:
# start a run
run_name = namesgenerator.get_random_name()
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(
    experiment_id, run_name, "pipeline.yaml",
    {
        "learning-rate": "0.01",
        "batch-size": "256",
        "epochs": "10",
        "hydrosphere-address": hydrosphere_address,
        "acceptable-accuracy": "0.90",
    }
)

Starting a new run with the name silly_varahamihira


# Deploy to Production

And finally deploying the model to production application

Build and publish image

In [20]:
%%bash 
docker build -t tidylobster/mnist-pipeline-deploy-to-prod:latest --no-cache 06_deploy-to-prod/ 
docker push tidylobster/mnist-pipeline-deploy-to-prod:latest


Step 1/4 : FROM tidylobster/odsc-base:1.0
 ---> a44d37e5b862
Step 2/4 : ADD ./execute.sh /src/
 ---> e285ac535611
Step 3/4 : WORKDIR /src/
 ---> Running in ddf7dac09750
Removing intermediate container ddf7dac09750
 ---> e4b7541b68cf
Step 4/4 : ENTRYPOINT [ "bash", "execute.sh" ]
 ---> Running in 2a479e1132e6
Removing intermediate container 2a479e1132e6
 ---> 84119c6bef76
Successfully built 84119c6bef76
Successfully tagged tidylobster/mnist-pipeline-deploy-to-prod:latest
The push refers to repository [docker.io/tidylobster/mnist-pipeline-deploy-to-prod]
537310a8c9b3: Preparing
66a75017de07: Preparing
83c720aa6f39: Preparing
56995c671038: Preparing
eee35c27cf87: Preparing
93ed1238773c: Preparing
eeb5ce6b3db4: Preparing
886601877ba4: Preparing
0fc100fdc7f9: Preparing
68dda0c9a8cd: Preparing
f67191ae09b8: Preparing
b2fd8b4c3da7: Preparing
0de2edf7bff4: Preparing
93ed1238773c: Waiting
f67191ae09b8: Waiting
b2fd8b4c3da7: Waiting
0de2edf7bff4: Waiting
eeb5ce6b3db4: Waiting
0fc100fdc7f9: Wait

### Pipeline

Define container operation

In [18]:
def deploy_to_prod_op(release, **kwargs):
    deploy_to_prod = dsl.ContainerOp(
        name="deploy_to_prod",
        image=f"{username}/mnist-pipeline-deploy-to-prod:latest",   # <-- Replace with correct docker image
        arguments=[
            "--model-version", release.outputs["model-version"],
            "--model-name", kwargs["model_name"],
            "--hydrosphere-address", kwargs["hydrosphere_address"]
        ],
    )

    return deploy_to_prod

In [19]:
@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(
    mount_path="/storage",
    learning_rate="0.01",
    epochs="10",
    batch_size="256",
    model_name="mnist",
    hydrosphere_address="",
    acceptable_accuracy="0.90",
):
    
    download = download_op(
        mount_path=mount_path)
    
    train = train_op(
        download, 
        mount_path=mount_path, 
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=batch_size)
    train.after(download)
    train.set_memory_request('1G')
    train.set_cpu_request('1')
    
    release = release_op(
        download, 
        train
        mount_path=mount_path, 
        hydrosphere_address=hydrosphere_address,
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=batch_size)
    release.after(train)
    
    deploy_to_stage = deploy_to_stage_op(
        release
        hydrosphere_address=hydrosphere_address,
        model_name=model_name)
    deploy_to_stage.after(release)
    
    test = test_op(
        download, 
        deploy_to_stage,
        mount_path=mount_path,
        hydrosphere_address=hydrosphere_address,
        acceptable_accuracy=acceptable_accuracy,
        model_name=model_name)
    test.set_retry(3)
    test.after(deploy_to_stage)
    
    deploy_to_prod = deploy_to_prod_op(
        release,
        model_name=model_name,
        hydrosphere_address=hydrosphere_address)
    deploy_to_prod.after(test)

In [20]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

In [21]:
%%bash 

tar -xvf pipeline.tar.gz
sed -i '' s/minio-service.kubeflow/minio-service.${NAMESPACE}/g pipeline.yaml
sed -i '' s/pipeline-runner/${NAMESPACE}-pipeline-runner/g pipeline.yaml

x pipeline.yaml


### Test

In [22]:
# start a run
run_name = namesgenerator.get_random_name()
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(
    experiment_id, run_name, "pipeline.yaml",
    {
        "learning-rate": "0.01",
        "batch-size": "256",
        "epochs": "10",
        "hydrosphere-address": hydrosphere_address,
        "acceptable-accuracy": "0.90",
    }
)

Starting a new run with the name vigorous_albattani


# Sample

Once we have a fully functional pipeline, we would like to automate running this pipeline. But we don't 

Build and publish image

In [49]:
%%bash 
docker build -t tidylobster/mnist-pipeline-sample:latest --no-cache 01_sample/ 
docker push tidylobster/mnist-pipeline-sample:latest


Step 1/4 : FROM tidylobster/odsc-base:1.0
 ---> a44d37e5b862
Step 2/4 : ADD ./sample.py /src/
 ---> bfb058e25199
Step 3/4 : WORKDIR /src/
 ---> Running in 29e23b657ee3
Removing intermediate container 29e23b657ee3
 ---> 2f4747efea6a
Step 4/4 : ENTRYPOINT [ "python", "sample.py" ]
 ---> Running in bbfe33fe0d33
Removing intermediate container bbfe33fe0d33
 ---> 3a2a04fe2c08
Successfully built 3a2a04fe2c08
Successfully tagged tidylobster/mnist-pipeline-sample:latest
The push refers to repository [docker.io/tidylobster/mnist-pipeline-sample]
5b283f22ec6c: Preparing
66a75017de07: Preparing
83c720aa6f39: Preparing
56995c671038: Preparing
eee35c27cf87: Preparing
93ed1238773c: Preparing
eeb5ce6b3db4: Preparing
886601877ba4: Preparing
0fc100fdc7f9: Preparing
68dda0c9a8cd: Preparing
f67191ae09b8: Preparing
b2fd8b4c3da7: Preparing
0de2edf7bff4: Preparing
886601877ba4: Waiting
93ed1238773c: Waiting
eeb5ce6b3db4: Waiting
0fc100fdc7f9: Waiting
68dda0c9a8cd: Waiting
f67191ae09b8: Waiting
b2fd8b4c3da7

### Pipeline

Define container operation

In [50]:
def sample_op():
    sample = dsl.ContainerOp(
        name="sample",
        image=f"{username}/mnist-pipeline-sample:latest", # <-- Replace with correct docker image
        file_outputs={"data_path": "/data_path.txt"}
        arguments=[
            "--mount-path", kwargs["mount_path"], 
            "--hydrosphere-address", kwargs["hydrosphere_address"],
        ]
    )  
    
    sample.add_volume(storage_volume)
    sample.add_volume_mount(storage_volume_mount)
    return sample

In [51]:
release@dsl.pipeline(name="mnist", description="MNIST classifier")
def pipeline_definition(
    mount_path="/storage",
    learning_rate="0.01",
    epochs="10",
    batch_size="256",
    model_name="mnist",
    hydrosphere_address="",
    acceptable_accuracy="0.90",
):
    
    sample = sample_op()
    
    train = train_op(
        sample, 
        mount_path=mount_path, 
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=batch_size)
    train.after(sample)
    train.set_memory_request('1G')
    train.set_cpu_request('1')
    
    release = release_op(
        sample, 
        train
        mount_path=mount_path, 
        hydrosphere_address=hydrosphere_address,
        learning_rate=learning_rate,
        epochs=epochs,
        batch_size=batch_size)
    release.after(train)
    
    deploy_to_stage = deploy_to_stage_op(
        release
        hydrosphere_address=hydrosphere_address,
        model_name=model_name)
    deploy_to_stage.after(release)
    
    test = test_op(
        download, 
        deploy_to_stage,
        mount_path=mount_path,
        hydrosphere_address=hydrosphere_address,
        acceptable_accuracy=acceptable_accuracy,
        model_name=model_name)
    test.set_retry(3)
    test.after(deploy_to_stage)
    
    deploy_to_prod = deploy_to_prod_op(
        release,
        model_name=model_name,
        hydrosphere_address=hydrosphere_address)
    deploy_to_prod.after(test)

In [52]:
compiler.Compiler().compile(pipeline_definition, "pipeline.tar.gz")

In [53]:
%%bash 

tar -xvf pipeline.tar.gz
sed -i '' s/minio-service.kubeflow/minio-service.${NAMESPACE}/g pipeline.yaml
sed -i '' s/pipeline-runner/${NAMESPACE}-pipeline-runner/g pipeline.yaml

x pipeline.yaml


### Test

In [54]:
# start a run
run_name = namesgenerator.get_random_name()
print("Starting a new run with the name {}".format(run_name))
result = client.run_pipeline(
    experiment_id, run_name, "pipeline.yaml",
    {
        "learning-rate": "0.01",
        "batch-size": "256",
        "epochs": "10",
        "hydrosphere-address": hydrosphere_address,
        "acceptable-accuracy": "0.90",
    }
)

Starting a new run with the name focused_spence
